In [105]:
#Importamos las librerias, están en el orden que fueron usadas

import pandas as pd
import operator
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures


#Los datos los obtuve de https://github.com/CSSEGISandData/COVID-19, Johns Hopkins University

#365 días que representan un año
days = 300

#Definimos la función logística
def f_logistic(x = 1, L = 2, a = 0, K = 3.7):
    return L/(1 + math.e**(-k(x - a)))

#Contrucción de los valores de la función logística
def constru_logistic_df(x_r = [], L = 2, a = 0, K = 3.7):
    logistic_df = pd.DataFrame(colums = ['x', 'y'])
    logistic_df['x'] = x_r
    logistic_df['y'] = [f_logistic(n, L, a, K) for n in r_x]
    return logistic_df

#Factores de crecimiento 
def  constru_growth_factors(y):
    return np.array([x / y[i - 1] for i, x in enumerate(y) if i > 0])

def constru_growth_factor_other(n, lim):
    x1 = 0
    y1 = 0.01
    factor_growth = 2
    x = 1
    
    while factor_growth > lim:
        y = (n * x) ** 2
        factor_growth = y / y1
        x = x + 1
        y = y1
    return x    

#Plot, curva

def curva(data, title):
    
    fig = go.Figure()
    for key in data.keys():
        df = data[key]
        fig.add_trace(go.Scatter(x = df['x'], y = df['y'], mode = 'lines', name = key))
        
        fig.update_layout(title_text = title)
        fig.show()

#Plot, exponencial

def expo_curve(start, stop, num):
    x_r = np.linspace(start, stop, num)
    df = pd.DataFrame(colums = ['x', 'y'])
    df['x'] = x_r
    df['y'] = [x2_funt(n) for n in x_r]
    
    y_min = df['y'].min()
    x_min = df.loc[df['y'] == df['y'].min()]['x'].min()
    
    x = -x_min
    y = -y_min
    
    df['x'] = df['x'].apply(lambda value: value + x)
    df['y'] = df['y'].apply(lambda value: value + y)
    
    df = df.loc[(df['x'] > 0) & (df['y'] > 0)]
    
    return df.reset_index(drop = True)

#Logaritmica

def log_exp(L = 2, a = 0, K=1): 
    x_r = np.linscape(0, days * 2, num = 2000)
    
    logistic_df = num_logistic_df(x_r, L = L, a = a, K = K)
    logistic_df = logistic_df.loc[(logistic_df['y'] >= 0.01)].reindex()
    
    return logistic_df

def logistic_func(L = 2, a = 0, K = 1):
    x_r = np.arange(-days,days)
    
    logistic_df = num_logistic_df(x_r, L = L, a = a, K = K)
    
    return logistic_df

#Mover plots desde el origen

def to_origen(df):
    
    y_min = df['y'].min()
    x_min = df.loc[df['y'] == df['y'].min()]['x'].min()
    
    x = -x_min
    y = -y_min
    
    df['x'] = df['x'].apply(lambda value: value + x)
    df['y'] = df['y'].apply(lambda value: value + y)
    
    df = df.loc[(df['x'] > 0) & (df['y'] > 0)]
    
    return df.reset_index(drop = True)
    
    
#model_selection 

def to_polinomyal(x):
    polynomial_features = PolynomialFeatures(degree = 2)
    x = x[:, np.newaxis]
    
    return polynomial_features.fit_transform(x)

#Construcción del modelo

def model(x, y):
    x = to_polinomyal(x)
    y = to_polinomyal(y)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 300)
    
    Model = LinearRegression()
    return Model.fit(X_train, y_train)

#Predicciones

def predicted(Model, x_predict):
    x_predict = to_polinomyal(x_predict)
    y_predict = Model.predict(x_predict)
    
    predicts = pd.DataFrame(columns = ['x', 'y'])
    predicts['x'] = x_predict[:, 1]
    predicts['y'] = y_predict[:, 1]
    
    return predicts
    

#Importar archivo .csv

dataset = pd.read_csv('C:/Users/juand/OneDrive/Escritorio/Predicción/all_data.csv')

dataset['Report_Date'] = pd.to_datetime(dataset['Report_Date'])
dataset[['Confirmed', 'Deaths', 'Recovered']] == dataset[['Confirmed', 'Deaths', 'Recovered']].fillna(0)
dataset['x'] = np.arange(0, dataset.shape[0])
dataset.tail()



,Report_Date,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,x
18,2020-10-20,NaN,Colombia,10/21/2020 4:24,974139,29272,876731,18
19,2020-10-21,NaN,Colombia,10/22/2020 4:24,981700,29464,884895,19
20,2020-10-22,NaN,Colombia,10/23/2020 4:24,990373,29637,893810,20
21,2020-10-23,NaN,Colombia,10/24/2020 4:24,998942,29802,901652,21
22,2020-10-24,NaN,Colombia,10/25/2020 4:24,1007711,30000,907379,22


In [72]:
#Casos por día

cases_day_df = dataset[['x', 'Confirmed']]
cases_day_df.rename(columns = {'Confirmed': 'y'}, inplace = True)

cases_day_df.loc[:, 'y'] = cases_day_df['y'].diff()

datos = {'Casos por día': cases_day_df}

curva(datos, 'Casos por día')

In [73]:
cases2_df = dataset[['x', 'Confirmed']]
cases2_df.rename(columns = {'Confirmed': 'y'}, inplace = True)

datos = {'Casos 2': cases2_df}
curva(datos, 'Casos 2')

In [96]:
#Funcion logística

def parabola(n, start, end, num):
    df = pd.DataFrame(columns = ['x', 'y'])
    df['x'] = np.linscape(start, edn, num)
    df['y'] = (n * df['x'] ** 2)
    return df

def logistic_final(L, a, K, start, end, num):
    x_r = np.linscape(start, end, num)
    logistic_df = num_logistic_df(x_r, L = L, a = a, K = K)
    
    return logistic_df

def areas(curva1, curva2):
    x_start_curva1 = curva1['x'].min()
    x_end_curva1 = curva1['x'].max()
    
    x_start_curva2 = curva2['x'].min()
    x_edn_curva2 = curva2['x'].max()
    
    x_start = x_start_curva1 if x_start_curva1 > x_start_curva2 else x_start_curva2
    x_end = x_end_curva1 if x_end_curva1 < x_end_curva2 else x_end_curva2
    
    curva1 = curva1.loc[(curva1['x'] > x_start) & (curva1['x'] < x_end)]
    curva2 = curva2.loc[(curva2['x'] > x_start) & (curva2['x'] < x_edn)]
    
    curva1_area = np.trapz(curva1['y'], curva1['x'])
    curva2_area = np.trapz(curva2['y'], curva2['x'])
    return np.abs(curva2_area - curva1_area)

def form_curva(data_points):
    area_results = {}
    start = data_points['x'].min()
    end = data_points['x'].max()
    num = end
    
    n_r = [x for x in np.arange(0, 5, 0.05)]
    
    for n in n_r:
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                data_points_2 = parabola(n, start, end, num)
                
                area = areas(data_points, data_points_2)
                
                area_results[str(n)] = str(area)
            except Warning:
                pass
    area_results_df = pd.DataFrame.from_dict(area_results, orient = 'index').reset_index()
    area_results_df = area_results_df.astype(float)
    area_results_df.columns = ['n', 'area']
    
    min_area = area_results_df['area'].min()
    min_area = area_results_df.loc[area_results_df['area'] == min_area]
    return min_area['n'].min()

def calculo_k(n, L, x_infle):
    parabola_seg = parabola(n, 1, x_infle, 100) / 5
    
    logistic_curva = []
    area_results = {}
    
    x_r = [x for x in np.arange(0, 2, 0.01)]
    
    for k in x_r:
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                log_exp = logistic_final(L = L, a = x_infle, K = K, start = 1, end = x_infle, num = 100)
                log_exp = log_exp / 5
                
                area = areas(log_exp, parabola_seg)
                logistic_curva += [(K, log_exp, area)]
                
                area_results[str(K)] = str(area)
            except Warning: 
                pass
            
    area_results_df = pd.DataFrame.from_dict(area_results, orient = 'index').reset_index(
    )
    area_results_df = area_results_df.astype(float)
    area_results_df.columns = ['n', 'area']
    
    min_area = area_results_df['area'].min()
    min_area = area_results_df.loc[area_results_df['area'] == min_area]
    
    return min_area['n'].min()

def calculo_k(n, L, x_infle):
    parabola_seg = parabola(n, 1, x_infle, 100) / 5
    logistic_curvas = []
    
    area_results = {}
    
    x_r = [x for x in np.arange(0, 2, 0.01)]
    for K in x_r:
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                log_exp = logistic_final(L = L, a = x_infle, K = K, start = 1, end = x_infle, num = 100)
                log_exp = log_exp / 5
                area = areas(log_exp, parabola_seg)
                
                logistic_curvas += [(K, log_exp, area)]
                
                area_results[str(K)] = str(area)
            except Warning:
                pass
            
    area_results_df = pd.DataFrame.from_dict(area_results, orient = 'index').reset_index()
    area_results_df = area_results_df.astype(float)
    area_results_df.columns = ['K', 'area']
    
    min_area = area_results_df['area'].min()
    min_area = area_results_df.loc[area_results_df['area'] == min_area]
    K = float(min_area['K'].min())
    return K

In [107]:
train_df = cases2_df[: cases2_df.shape[0]]
Model = model(train_df['x'], train_df['y'])
predictions = predicted(Model, np.arange(0, days))

fig = go.Figure()
fig.add_trace(go.Scatter(x = cases2_df['x'], y = cases2_df['y'], mode = 'lines+markers', name = 'Casos confirmados'))
fig.add_trace(go.Scatter(x = predictions['x'], y = predictions['y'], mode = 'lines', name = 'Pronóstico'))

fig.update_layout(title_text = "Casos confirmados vs predicciones")
fig.show()

In [110]:
#Punto de inflección

steps = predictions['y']
factor = np.array([x / steps[i - 1] for i, x in enumerate(steps) if i > 0])
factor = np.concatenate((np.array([0]), factor))
predictions["factor"] = factor
peaks, _ = find_peaks(predictions['factor'], height = 0)

fig = go.Figure()

fig.add_trace(go.Scatter(x = predictions['x'], y = predictions['factor'], mode = 'lines', name = ""))

fig.update_layout(title_text= "Factor de crcecimiento")

fig.show()
print(peaks)

[  1 280]


In [112]:
inflection = predictions[predictions['factor'] > 1].tail()
display(inflection)
inflection_idx = inflection.index[0]
inflection_x = inflection['x'].max()
inflection_y = inflection['y'].max()
L = inflection_y * 2

print(f'inflection_idx: {inflection_idx}')
print(f'inflection_x: {inflection_x}')
print(f'inflection_y: {inflection_y}')
print(f'L: {L}')

,x,y,factor
295,295.0,-261887.832664,1.066129
296,296.0,-278217.374732,1.062353
297,297.0,-294632.275975,1.059000
298,298.0,-311132.536394,1.056003
299,299.0,-327718.155989,1.053307


inflection_idx: 295
inflection_x: 299.0
inflection_y: -261887.8326640583
L: -523775.6653281166


In [114]:
pred = predictions.iloc[:inflection_idx + 1]
fig = go.Figure()
fig.add_trace(go.Scatter(x = pred['x'], y = pred['y'], line = dict(dash = 'dash'), name = 'Pronóstico'))

fig.update_layout()
fig.show()

In [116]:
predictions_inv = predictions[:inflection_idx + 1][::-1].copy()
predictions_b = predictions_inv.copy()

predictions_b['x'] = np.arange(inflection_idx, inflection_idx*2+1)
predictions_b['y'] = predictions_b['y'].apply(lambda y: L - y)

full_predictions = predictions[:inflection_idx]
full_predictions = full_predictions.append(predictions_b)

cases2_df.loc[:,'date'] = pd.date_range(start='3/6/2020', periods=cases2_df.shape[0])
full_predictions.loc[:,'date'] = pd.date_range(start='3/6/2020', periods=full_predictions.shape[0])

fig = go.Figure()
fig.add_trace(go.Scatter(x=cases2_df['date'], y=cases2_df['y'], mode='lines+markers', line=dict(width=1), name='Casos confirmados'))
fig.add_trace(go.Scatter(x=full_predictions['date'], y=full_predictions['y'], line=dict(dash='dash'), name = 'Pronóstico'))

fig.update_layout(title_text="Casos confirmados vs Pronósticos")

fig.show()